In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Read In Data

In [52]:
df_BA=pd.read_csv('BA')
df_DS=pd.read_csv('DS')
df_DA=pd.read_csv('DA')
df_DE=pd.read_csv('DE')

# Sample and Split

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
len(df_DS),len(df_DA),len(df_BA),len(df_DE)

(973, 3983, 1663, 1167)

In [55]:
df_DA=df_DA.sample(n=1000,replace=True)
df_BA=df_BA.sample(n=1000,replace=True)

In [56]:
len(df_DS),len(df_DA),len(df_BA),len(df_DE)

(973, 1000, 1000, 1167)

In [57]:
whole_list=[df_DA,df_DS,df_DE,df_BA]
df_whole=pd.concat(whole_list)

In [58]:
df_whole

,JT,JD,tool_list,skill_list,degree_list,clean
2172,DA,The Pharmacy Data Analyst will be responsible ...,['excel'],"['analyz', 'end user', 'data', 'account', 'com...",1.0,"['excel', 'analyz', 'end user', 'data', 'accou..."
2782,DA,Bigger challenges.\nBolder ideas.\nGlobal impa...,"['seaborn', 'scikit', 'matplotlib', 'python', ...","['visual', 'regress', 'problem solving', 'mach...",1.0,"['seaborn', 'scikit', 'matplotlib', 'python', ..."
1301,DA,Req ID: 94674\nRole Responsibilities:\n* Estab...,[None],"['busi', 'etl', 'healthcar']",1.0,"['busi', 'etl', 'healthcar']"
1150,DA,Data Analyst III\nDuration: 6+ months\nDuties:...,"['c', 'sql', 'power bi', 'bi', 'excel', 'table...","['visual', 'problem solving', 'data', 'busi', ...",1.0,"['c', 'sql', 'powerbi', 'excel', 'tableau', 'v..."
1327,DA,"Hello,\nÂ\nHope you are doing goodâ!!\n!Â\nÂ\n...",['sql'],['data'],999.0,"['sql', 'data']"
...,...,...,...,...,...,...
1429,NaN,Job title: Salesforce BSA with Service Cloud a...,"['cloud', 'salesforc']",['busi'],999.0,"['cloud', 'salesforc', 'busi']"
281,NaN,Req ID: 177704\nThe Job\nWarner Bros. Entertai...,['hana'],"['regress', 'busi', 'commun', 'optim']",999.0,"['hana', 'regress', 'busi', 'commun', 'optim']"
1595,NaN,You will be the go-to individual for all Web D...,"['excel', 'microsoft', 'javascript']","['research', 'busi', 'commun']",999.0,"['excel', 'microsoft', 'javascript', 'research..."
899,NaN,Job Summary:\nLooking for high powered and qua...,[None],"['data', 'busi', 'statist']",2.0,"['data', 'busi', 'statist']"


In [59]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_whole, test_size = 0.3)
clean_train=df_train["clean"]
Y_train=df_train["JT"]
clean_test=df_test["clean"]
Y_test=df_test["JT"]

# Decision Tree/ Random Forest

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.tree import DecisionTreeClassifier

In [30]:
vectorizer = CountVectorizer().fit(df_whole["clean"])
clean_bow= vectorizer.transform(df_whole["clean"])
train_bow = vectorizer.transform(clean_train)
test_bow = vectorizer.transform(clean_test)

In [31]:
tfidf_transformer = TfidfTransformer().fit(clean_bow)
X_train = tfidf_transformer.transform(train_bow)
X_test = tfidf_transformer.transform(test_bow)
print(X_train.shape)

(2898, 226)


In [32]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train,Y_train)

DecisionTreeClassifier()

In [33]:
predictions = dtree.predict(X_test)
print(classification_report(Y_test,predictions))

              precision    recall  f1-score   support

          BA       0.76      0.86      0.80       308
          DA       0.64      0.64      0.64       289
          DE       0.82      0.77      0.79       358
          DS       0.78      0.73      0.76       287

    accuracy                           0.75      1242
   macro avg       0.75      0.75      0.75      1242
weighted avg       0.75      0.75      0.75      1242



In [35]:
#conda install -c conda-forge skater

In [36]:
from sklearn.pipeline import Pipeline

pipelines = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', DecisionTreeClassifier()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [37]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from skater.model import InMemoryModel
from skater.core.explanations import Interpretation
transfromed_x_test = vectorizer.transform(clean_train).toarray()
interpreter = Interpretation(transfromed_x_test, feature_names=vectorizer.get_feature_names())
for pipeline in pipelines:
    print('-' * 50)
    #print('Estimator: %s' % (pipeline['name']))
    
    if pipeline['name'] in ['Logistic Regression', 'Random Forest']:
        estimator = pipeline['pipeline'].steps[1][1]
    else:
        continue
        
    print(estimator)
        
    pyint_model = InMemoryModel(estimator.predict_proba, examples=transfromed_x_test)
    
    f, axes = plt.subplots(1, 1, figsize = (26, 18))
    ax = axes
    interpreter.feature_importance.plot_feature_importance(pyint_model, ascending=False, ax=ax)
    plt.show()

ModuleNotFoundError: No module named 'skater'